In [1]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

In [2]:
yf.pdr_override()


def cheap_opts(symbol, calls_or_puts):
  '''
  This searches all possible expiry dates and finds contracts.
  '''
  import sys, os
  from datetime import date

  #We're going to suppress prints b/c datareader is annoying then restore printing so this helps
  #old_stdout = sys.stdout
  #sys.stdout = open(os.devnull, "w")

  finaldf = pd.DataFrame()
  ticker = yf.Ticker(symbol)
  stock_price = data.get_data_yahoo(symbol, end_date = date.today())['Close'][-1]
  for opt_date in ticker.options:
    opt = ticker.option_chain(opt_date)
    if calls_or_puts == 'puts':
      opt.puts.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.puts)
    else:
      opt.calls.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.calls)
  return finaldf, stock_price

In [3]:
calls_or_puts = 'calls'
symbol = 'tsla'

results, stock_price = cheap_opts(symbol, calls_or_puts)
returned = results
# Do some final formatting changes
returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
if calls_or_puts == 'puts':
    returned.insert(3, 'dist OTM', stock_price - returned['strike'])
else:
    returned.insert(3, 'dist OTM', returned['strike'] - stock_price)

returned.insert(4, '% OTM', returned['dist OTM']/stock_price*100)
returned['value'] = returned['openInterest']*returned['lastPrice']*100

# Do some filtering
returned = returned[returned['% OTM'] < 35]
returned = returned[returned['lastPrice'] < 15]
#returned = returned[returned['volume'] > 10]
returned = returned[returned['opt_date'] > '2020-04-24']
#returned = returned[returned['percentChange'] < -0]

# Sort and display
returned.sort_values(inplace=True, by='lastPrice', ascending=True)
#returned = returned.head(30)
returned.reset_index(drop=True, inplace=True)
final = returned.style.set_precision(3).set_caption('<H1>' + symbol + ' ' + calls_or_puts.upper() + \
                                          '   (Current = $' + str(round(stock_price, 2)) + ')')

[*********************100%***********************]  1 of 1 completed


In [7]:
# Download the CSV and you're done!
filename = 'cheap_' + symbol + '_' + calls_or_puts + '.csv'

returned.to_csv(filename)
returned.to_csv('unusual_calls_activity_5_15_exp.csv')

In [11]:
top = returned.sort_values(by='lastPrice', ascending=True) #.head(70)
top.sort_values(by='% OTM', inplace=True, ascending=True)
a = top.style.set_caption(calls_or_puts.upper())
a
# top[top['inTheMoney']== True]

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
14,2020-05-01,850,13.4,124.85,17.2171,12.8,13.5,-0.64,-4.55841,893,669,1.24891,False,896460
11,2020-05-01,860,11.75,134.85,18.5961,11.45,12.4,-0.85,-6.74603,296,120,1.25891,False,141000
10,2020-05-01,870,10.75,144.85,19.9752,10.3,10.95,-0.85,-7.32759,473,256,1.2613,False,275200
7,2020-05-01,880,9.8,154.85,21.3542,9.3,10,-1,-9.25926,169,229,1.27173,False,224420
5,2020-05-01,890,8.75,164.85,22.7332,8.35,8.9,-1.11,-11.2576,146,85,1.27576,False,74375
4,2020-05-01,900,7.9,174.85,24.1122,7.7,8,-1.16,-12.8035,1779,1039,1.2865,False,820810
18,2020-05-08,900,14.5,174.85,24.1122,13.9,15.25,-0.25,-1.69492,111,168,1.10529,False,243600
3,2020-05-01,920,6.85,194.85,26.8703,6.1,6.85,-0.8,-10.4575,268,231,1.30555,False,158235
12,2020-05-08,920,12.4,194.85,26.8703,11.85,13.15,-0.6,-4.61539,62,44,1.11676,False,54560
9,2020-05-08,940,10.58,214.85,29.6283,10.15,11.45,-2.22,-17.3438,9,33,1.13013,False,34914


In [16]:
#top.to_csv('filtered_puts_05_15.csv')
top.to_csv('filtered_'+calls_or_puts+'_05_15.csv')

otm=returned['inTheMoney'] == False
returned.loc[otm,'Distance OTM']=999-returned.lastPrice
returned

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value,Distance OTM
0,2020-05-01,960.0,4.49,234.849976,32.386398,4.20,4.85,-1.150000,-20.390072,107.0,125.0,1.348270,False,56125.0,994.51
1,2020-05-01,950.0,4.90,224.849976,31.007373,4.90,5.05,-1.100000,-18.333332,862.0,467.0,1.339847,False,228830.0,994.10
2,2020-05-01,940.0,5.34,214.849976,29.628348,5.05,5.75,-1.210000,-18.473282,188.0,201.0,1.326908,False,107334.0,993.66
3,2020-05-01,920.0,6.85,194.849976,26.870298,6.10,6.85,-0.800000,-10.457519,268.0,231.0,1.305545,False,158235.0,992.15
4,2020-05-01,900.0,7.90,174.849976,24.112248,7.70,8.00,-1.160000,-12.803535,1779.0,1039.0,1.286503,False,820810.0,991.10
5,2020-05-01,890.0,8.75,164.849976,22.733223,8.35,8.90,-1.110000,-11.257604,146.0,85.0,1.275760,False,74375.0,990.25
6,2020-05-08,960.0,9.40,234.849976,32.386398,8.80,10.00,-0.600000,-6.000004,30.0,47.0,1.144902,False,44180.0,989.60
7,2020-05-01,880.0,9.80,154.849976,21.354198,9.30,10.00,-1.000000,-9.259259,169.0,229.0,1.271732,False,224420.0,989.20
8,2020-05-08,950.0,10.04,224.849976,31.007373,9.70,10.15,-0.610000,-5.727696,39.0,93.0,1.132756,False,93372.0,988.96
9,2020-05-08,940.0,10.58,214.849976,29.628348,10.15,11.45,-2.220000,-17.343752,9.0,33.0,1.130131,False,34914.0,988.42


In [ ]:
# returned.style.set_precision(2)
returned
#top[top['inTheMoney']== True]